In [ ]:
import alea.sim
import numpy as np

from alea.sim.epa.earth_magnetic_field import EarthMagneticFieldModel
from alea.sim.epa.attitude_dynamics import AttitudeDynamicsModel
from alea.sim.epa.orbit_dynamics import OrbitDynamicsModel
from alea.sim.kernel.kernel import AleasimKernel

from alea.sim.spacecraft.sensors import SimpleMagSensor
from alea.sim.spacecraft.sensors import SimpleSunSensor
from alea.sim.spacecraft.sensors import GyroSensor
from alea.sim.algorithms.filters.low_pass_filter import LowPassFilter
import matplotlib.pyplot as plt

## Moving average filter examples

In [ ]:
signal_length = 500
sample_period = 0.01
window_size = 5

In [ ]:
# Magnetometer filtering example

kernel = AleasimKernel(dt=sample_period)
magm = EarthMagneticFieldModel(kernel)
odyn = OrbitDynamicsModel(kernel)
magsense = SimpleMagSensor('mag_sensor', kernel, int(1 / sample_period))
mag_filter = LowPassFilter('avg', window_size=window_size)

kernel.add_model(magm)
kernel.add_model(odyn)
kernel.add_model(magsense, create_shared_mem=True)

magsense.power_on()

gt_signal = []
noisy_signal = []
filtered_signal = []

for i in range(signal_length):
    # Advance the kernel
    kernel.step()
    magsense.measure()

    # Get the latest x, y, z readings from the state array
    gt_signal_temp = magsense.state_array[-1, :3]
    noisy_signal_temp = magsense.state_array[-1, 3:6]
    filered_signal_temp = mag_filter.filter_output(magsense.state_array[:, 3:6])

    # Store the signals
    gt_signal.append(gt_signal_temp)
    noisy_signal.append(noisy_signal_temp)
    filtered_signal.append(filered_signal_temp)

# Convert lists to numpy arrays for easier plotting
gt_signal = np.array(gt_signal)
noisy_signal = np.array(noisy_signal)
filtered_signal = np.array(filtered_signal)

time = np.arange(signal_length) * sample_period

# noisy_signal = noisy_signal[10:]
# filtered_signal = filtered_signal[10:]
# time = time[10:]

# Plot noisy and filtered signals for each axis (x, y, z)
plt.figure(figsize=(12, 6))

# Plot x-axis signals
plt.subplot(3, 1, 1)
plt.plot(time, gt_signal[:, 0], label='True signal', alpha=0.6)
plt.plot(time, noisy_signal[:, 0], label='Noisy signal', alpha=0.6)
plt.plot(time, filtered_signal[:, 0], label='Filtered signal', alpha=0.8)
plt.title('X-axis magnetometer signal')
plt.xlabel('Time (s)')
plt.legend()

# Plot y-axis signals
plt.subplot(3, 1, 2)
plt.plot(time, gt_signal[:, 1], label='True signal', alpha=0.6)
plt.plot(time, noisy_signal[:, 1], label='Noisy signal', alpha=0.6)
plt.plot(time, filtered_signal[:, 1], label='Filtered signal', alpha=0.8)
plt.title('Y-axis magnetometer signal')
plt.xlabel('Time (s)')
plt.legend()

# Plot z-axis signals
plt.subplot(3, 1, 3)
plt.plot(time, gt_signal[:, 2], label='True signal', alpha=0.6)
plt.plot(time, noisy_signal[:, 2], label='Noisy signal', alpha=0.6)
plt.plot(time, filtered_signal[:, 2], label='Filtered signal', alpha=0.8)
plt.title('Z-axis magnetometer signal')
plt.xlabel('Time (s)')
plt.legend()

# Adjust layout and show the plot
plt.tight_layout()
plt.show()

In [ ]:
# Sun sensor filtering example

kernel = AleasimKernel(dt=sample_period)
odyn = OrbitDynamicsModel(kernel)
sunsense = SimpleSunSensor('sun_sensor', kernel, int(1 / sample_period))
sun_filter = LowPassFilter('avg', window_size=window_size)

kernel.add_model(odyn)
kernel.add_model(sunsense, create_shared_mem=True)

sunsense.power_on()

gt_signal = []
noisy_signal = []
filtered_signal = []

for i in range(signal_length):
    # Advance the kernel
    kernel.step()
    sunsense.measure()

    # Get the latest x, y, z readings from the state array
    gt_signal_temp = sunsense.state_array[-1, :3]
    noisy_signal_temp = sunsense.state_array[-1, 3:6]
    filered_signal_temp = mag_filter.filter_output(sunsense.state_array[:, 3:6])

    # Store the signals
    gt_signal.append(gt_signal_temp)
    noisy_signal.append(noisy_signal_temp)
    filtered_signal.append(filered_signal_temp)

# Convert lists to numpy arrays for easier plotting
gt_signal = np.array(gt_signal)
noisy_signal = np.array(noisy_signal)
filtered_signal = np.array(filtered_signal)

time = np.arange(signal_length) * sample_period

# noisy_signal = noisy_signal[10:]
# filtered_signal = filtered_signal[10:]
# time = time[10:]

# Plot noisy and filtered signals for each axis (x, y, z)
plt.figure(figsize=(12, 6))

# Plot x-axis signals
plt.subplot(3, 1, 1)
plt.plot(time, gt_signal[:, 0], label='True signal', alpha=0.6)
plt.plot(time, noisy_signal[:, 0], label='Noisy signal', alpha=0.6)
plt.plot(time, filtered_signal[:, 0], label='Filtered signal', alpha=0.8)
plt.title('X-axis sun sensor signal')
plt.xlabel('Time (s)')
plt.legend()

# Plot y-axis signals
plt.subplot(3, 1, 2)
plt.plot(time, gt_signal[:, 1], label='True signal', alpha=0.6)
plt.plot(time, noisy_signal[:, 1], label='Noisy signal', alpha=0.6)
plt.plot(time, filtered_signal[:, 1], label='Filtered signal', alpha=0.8)
plt.title('Y-axis sun sensor signal')
plt.xlabel('Time (s)')
plt.legend()

# Plot z-axis signals
plt.subplot(3, 1, 3)
plt.plot(time, gt_signal[:, 2], label='True signal', alpha=0.6)
plt.plot(time, noisy_signal[:, 2], label='Noisy signal', alpha=0.6)
plt.plot(time, filtered_signal[:, 2], label='Filtered signal', alpha=0.8)
plt.title('Z-axis sun sensor signal')
plt.xlabel('Time (s)')
plt.legend()

# Adjust layout and show the plot
plt.tight_layout()
plt.show()

In [ ]:
# Gyro filtering example

kernel = AleasimKernel(dt=sample_period)
adyn = AttitudeDynamicsModel(kernel)
gyro = GyroSensor('gyro_sensor', kernel, int(1 / sample_period))
gyro_filter = LowPassFilter('avg', window_size=window_size)

kernel.add_model(adyn)
kernel.add_model(gyro, create_shared_mem=True)

gyro.power_on()

gt_signal = []
noisy_signal = []
filtered_signal = []

for i in range(signal_length):
    # Advance the kernel
    kernel.step()
    gyro.measure()

    # Get the latest x, y, z readings from the state array
    gt_signal_temp = gyro.state_array[-1, :3]
    noisy_signal_temp = gyro.state_array[-1, 3:6]
    filered_signal_temp = gyro_filter.filter_output(gyro.state_array[:, 3:6])

    # Store the signals
    gt_signal.append(gt_signal_temp)
    noisy_signal.append(noisy_signal_temp)
    filtered_signal.append(filered_signal_temp)

# Convert lists to numpy arrays for easier plotting
gt_signal = np.array(gt_signal)
noisy_signal = np.array(noisy_signal)
filtered_signal = np.array(filtered_signal)

time = np.arange(signal_length) * sample_period

# noisy_signal = noisy_signal[10:]
# filtered_signal = filtered_signal[10:]
# time = time[10:]

# Plot noisy and filtered signals for each axis (x, y, z)
plt.figure(figsize=(12, 6))

# Plot x-axis signals
plt.subplot(3, 1, 1)
plt.plot(time, gt_signal[:, 0], label='True signal', alpha=0.6)
plt.plot(time, noisy_signal[:, 0], label='Noisy signal', alpha=0.6)
plt.plot(time, filtered_signal[:, 0], label='Filtered signal', alpha=0.8)
plt.title('X-axis gyro signal')
plt.legend()
plt.xlabel('Time (s)')

# Plot y-axis signals
plt.subplot(3, 1, 2)
plt.plot(time, gt_signal[:, 1], label='True signal', alpha=0.6)
plt.plot(time, noisy_signal[:, 1], label='Noisy signal', alpha=0.6)
plt.plot(time, filtered_signal[:, 1], label='Filtered signal', alpha=0.8)
plt.title('Y-axis gyro signal')
plt.xlabel('Time (s)')
plt.legend()

# Plot z-axis signals
plt.subplot(3, 1, 3)
plt.plot(time, gt_signal[:, 2], label='True signal', alpha=0.6)
plt.plot(time, noisy_signal[:, 2], label='Noisy signal', alpha=0.6)
plt.plot(time, filtered_signal[:, 2], label='Filtered signal', alpha=0.8)
plt.title('Z-axis gyro signal')
plt.xlabel('Time (s)')
plt.legend()

# Adjust layout and show the plot
plt.tight_layout()
plt.show()

## Butterworth filter examples

In [ ]:
order = 4
cutoff_freq = 10

In [ ]:
# Magnetometer filtering example

kernel = AleasimKernel(dt=sample_period)
magm = EarthMagneticFieldModel(kernel)
odyn = OrbitDynamicsModel(kernel)
magsense = SimpleMagSensor('mag_sensor', kernel, int(1 / sample_period))
mag_filter = LowPassFilter('butter', int(1 / sample_period), order=order, cutoff_freq=cutoff_freq)

kernel.add_model(magm)
kernel.add_model(odyn)
kernel.add_model(magsense, create_shared_mem=True)

magsense.power_on()

noisy_signal = []
filtered_signal = []

for i in range(signal_length):
    # Advance the kernel
    kernel.step()
    magsense.measure()

    # Get the latest x, y, z readings from the state array
    noisy_signal_temp = magsense.state_array[-1, 3:6]
    filered_signal_temp = mag_filter.filter_output(magsense.state_array[:, 3:6])

    # Store the signals
    noisy_signal.append(noisy_signal_temp)
    filtered_signal.append(filered_signal_temp)

# Convert lists to numpy arrays for easier plotting
noisy_signal = np.array(noisy_signal)
filtered_signal = np.array(filtered_signal)

time = np.arange(signal_length) * sample_period

# noisy_signal = noisy_signal[10:]
# filtered_signal = filtered_signal[10:]
# time = time[10:]

# Plot noisy and filtered signals for each axis (x, y, z)
plt.figure(figsize=(12, 6))

# Plot x-axis signals
plt.subplot(3, 1, 1)
plt.plot(time, noisy_signal[:, 0], label='Noisy signal', alpha=0.6)
plt.plot(time, filtered_signal[:, 0], label='Filtered signal', alpha=0.8)
plt.title('X-axis magnetometer signal')
plt.xlabel('Time (s)')
plt.legend()

# Plot y-axis signals
plt.subplot(3, 1, 2)
plt.plot(time, noisy_signal[:, 1], label='Noisy signal', alpha=0.6)
plt.plot(time, filtered_signal[:, 1], label='Filtered signal', alpha=0.8)
plt.title('Y-axis magnetometer signal')
plt.xlabel('Time (s)')
plt.legend()

# Plot z-axis signals
plt.subplot(3, 1, 3)
plt.plot(time, noisy_signal[:, 2], label='Noisy signal', alpha=0.6)
plt.plot(time, filtered_signal[:, 2], label='Filtered signal', alpha=0.8)
plt.title('Z-axis magnetometer signal')
plt.xlabel('Time (s)')
plt.legend()

# Adjust layout and show the plot
plt.tight_layout()
plt.show()

In [ ]:
# Sun sensor filtering example

kernel = AleasimKernel(dt=sample_period)
odyn = OrbitDynamicsModel(kernel)
sunsense = SimpleSunSensor('sun_sensor', kernel, int(1 / sample_period))
sun_filter = LowPassFilter('butter', int(1 / sample_period), order=order, cutoff_freq=cutoff_freq)

kernel.add_model(odyn)
kernel.add_model(sunsense, create_shared_mem=True)

sunsense.power_on()

noisy_signal = []
filtered_signal = []

for i in range(signal_length):
    # Advance the kernel
    kernel.step()
    sunsense.measure()

    # Get the latest x, y, z readings from the state array
    noisy_signal_temp = sunsense.state_array[-1, 3:6]
    filered_signal_temp = mag_filter.filter_output(sunsense.state_array[:, 3:6])

    # Store the signals
    noisy_signal.append(noisy_signal_temp)
    filtered_signal.append(filered_signal_temp)

# Convert lists to numpy arrays for easier plotting
noisy_signal = np.array(noisy_signal)
filtered_signal = np.array(filtered_signal)

time = np.arange(signal_length) * sample_period

# noisy_signal = noisy_signal[10:]
# filtered_signal = filtered_signal[10:]
# time = time[10:]

# Plot noisy and filtered signals for each axis (x, y, z)
plt.figure(figsize=(12, 6))

# Plot x-axis signals
plt.subplot(3, 1, 1)
plt.plot(time, noisy_signal[:, 0], label='Noisy signal', alpha=0.6)
plt.plot(time, filtered_signal[:, 0], label='Filtered signal', alpha=0.8)
plt.title('X-axis sun sensor signal')
plt.xlabel('Time (s)')
plt.legend()

# Plot y-axis signals
plt.subplot(3, 1, 2)
plt.plot(time, noisy_signal[:, 1], label='Noisy signal', alpha=0.6)
plt.plot(time, filtered_signal[:, 1], label='Filtered signal', alpha=0.8)
plt.title('Y-axis sun sensor signal')
plt.xlabel('Time (s)')
plt.legend()

# Plot z-axis signals
plt.subplot(3, 1, 3)
plt.plot(time, noisy_signal[:, 2], label='Noisy signal', alpha=0.6)
plt.plot(time, filtered_signal[:, 2], label='Filtered signal', alpha=0.8)
plt.title('Z-axis sun sensor signal')
plt.xlabel('Time (s)')
plt.legend()

# Adjust layout and show the plot
plt.tight_layout()
plt.show()

In [ ]:
# Gyro filtering example

kernel = AleasimKernel(dt=sample_period)
adyn = AttitudeDynamicsModel(kernel)
gyro = GyroSensor('gyro_sensor', kernel, int(1 / sample_period))
gyro_filter = LowPassFilter('butter', int(1 / sample_period), order=order, cutoff_freq=cutoff_freq)

kernel.add_model(adyn)
kernel.add_model(gyro, create_shared_mem=True)

gyro.power_on()

noisy_signal = []
filtered_signal = []

for i in range(signal_length):
    # Advance the kernel
    kernel.step()
    gyro.measure()

    # Get the latest x, y, z readings from the state array
    noisy_signal_temp = gyro.state_array[-1, 3:6]
    filered_signal_temp = gyro_filter.filter_output(gyro.state_array[:, 3:6])

    # Store the signals
    noisy_signal.append(noisy_signal_temp)
    filtered_signal.append(filered_signal_temp)

# Convert lists to numpy arrays for easier plotting
noisy_signal = np.array(noisy_signal)
filtered_signal = np.array(filtered_signal)

time = np.arange(signal_length) * sample_period

# noisy_signal = noisy_signal[:-20]
# filtered_signal = filtered_signal[:-20]
# time = time[:-20]

# Plot noisy and filtered signals for each axis (x, y, z)
plt.figure(figsize=(12, 6))

# Plot x-axis signals
plt.subplot(3, 1, 1)
plt.plot(time, noisy_signal[:, 0], label='Noisy signal', alpha=0.6)
plt.plot(time, filtered_signal[:, 0], label='Filtered signal', alpha=0.8)
plt.title('X-axis gyro signal')
plt.xlabel('Time (s)')
plt.legend()

# Plot y-axis signals
plt.subplot(3, 1, 2)
plt.plot(time, noisy_signal[:, 1], label='Noisy signal', alpha=0.6)
plt.plot(time, filtered_signal[:, 1], label='Filtered signal', alpha=0.8)
plt.title('Y-axis gyro signal')
plt.xlabel('Time (s)')
plt.legend()

# Plot z-axis signals
plt.subplot(3, 1, 3)
plt.plot(time, noisy_signal[:, 2], label='Noisy signal', alpha=0.6)
plt.plot(time, filtered_signal[:, 2], label='Filtered signal', alpha=0.8)
plt.title('Z-axis gyro signal')
plt.xlabel('Time (s)')
plt.legend()

# Adjust layout and show the plot
plt.tight_layout()
plt.show()

## Bessel filter examples

In [ ]:
# Magnetometer filtering example

kernel = AleasimKernel(dt=sample_period)
magm = EarthMagneticFieldModel(kernel)
odyn = OrbitDynamicsModel(kernel)
magsense = SimpleMagSensor('mag_sensor', kernel, int(1 / sample_period))
mag_filter = LowPassFilter('bessel', int(1 / sample_period), order=order, cutoff_freq=cutoff_freq)

kernel.add_model(magm)
kernel.add_model(odyn)
kernel.add_model(magsense, create_shared_mem=True)

magsense.power_on()

noisy_signal = []
filtered_signal = []

for i in range(signal_length):
    # Advance the kernel
    kernel.step()
    magsense.measure()

    # Get the latest x, y, z readings from the state array
    noisy_signal_temp = magsense.state_array[-1, 3:6]
    filered_signal_temp = mag_filter.filter_output(magsense.state_array[:, 3:6])

    # Store the signals
    noisy_signal.append(noisy_signal_temp)
    filtered_signal.append(filered_signal_temp)

# Convert lists to numpy arrays for easier plotting
noisy_signal = np.array(noisy_signal)
filtered_signal = np.array(filtered_signal)

time = np.arange(signal_length) * sample_period

# noisy_signal = noisy_signal[10:]
# filtered_signal = filtered_signal[10:]
# time = time[10:]

# Plot noisy and filtered signals for each axis (x, y, z)
plt.figure(figsize=(12, 6))

# Plot x-axis signals
plt.subplot(3, 1, 1)
plt.plot(time, noisy_signal[:, 0], label='Noisy signal', alpha=0.6)
plt.plot(time, filtered_signal[:, 0], label='Filtered signal', alpha=0.8)
plt.title('X-axis magnetometer signal')
plt.xlabel('Time (s)')
plt.legend()

# Plot y-axis signals
plt.subplot(3, 1, 2)
plt.plot(time, noisy_signal[:, 1], label='Noisy signal', alpha=0.6)
plt.plot(time, filtered_signal[:, 1], label='Filtered signal', alpha=0.8)
plt.title('Y-axis magnetometer signal')
plt.xlabel('Time (s)')
plt.legend()

# Plot z-axis signals
plt.subplot(3, 1, 3)
plt.plot(time, noisy_signal[:, 2], label='Noisy signal', alpha=0.6)
plt.plot(time, filtered_signal[:, 2], label='Filtered signal', alpha=0.8)
plt.title('Z-axis magnetometer signal')
plt.xlabel('Time (s)')
plt.legend()

# Adjust layout and show the plot
plt.tight_layout()
plt.show()

In [ ]:
# Sun sensor filtering example

kernel = AleasimKernel(dt=sample_period)
odyn = OrbitDynamicsModel(kernel)
sunsense = SimpleSunSensor('sun_sensor', kernel, int(1 / sample_period))
sun_filter = LowPassFilter('bessel', int(1 / sample_period), order=order, cutoff_freq=cutoff_freq)

kernel.add_model(odyn)
kernel.add_model(sunsense, create_shared_mem=True)

sunsense.power_on()

noisy_signal = []
filtered_signal = []

for i in range(signal_length):
    # Advance the kernel
    kernel.step()
    sunsense.measure()

    # Get the latest x, y, z readings from the state array
    noisy_signal_temp = sunsense.state_array[-1, 3:6]
    filered_signal_temp = mag_filter.filter_output(sunsense.state_array[:, 3:6])

    # Store the signals
    noisy_signal.append(noisy_signal_temp)
    filtered_signal.append(filered_signal_temp)

# Convert lists to numpy arrays for easier plotting
noisy_signal = np.array(noisy_signal)
filtered_signal = np.array(filtered_signal)

time = np.arange(signal_length) * sample_period

# noisy_signal = noisy_signal[10:]
# filtered_signal = filtered_signal[10:]
# time = time[10:]

# Plot noisy and filtered signals for each axis (x, y, z)
plt.figure(figsize=(12, 6))

# Plot x-axis signals
plt.subplot(3, 1, 1)
plt.plot(time, noisy_signal[:, 0], label='Noisy signal', alpha=0.6)
plt.plot(time, filtered_signal[:, 0], label='Filtered signal', alpha=0.8)
plt.title('X-axis sun sensor signal')
plt.xlabel('Time (s)')
plt.legend()

# Plot y-axis signals
plt.subplot(3, 1, 2)
plt.plot(time, noisy_signal[:, 1], label='Noisy signal', alpha=0.6)
plt.plot(time, filtered_signal[:, 1], label='Filtered signal', alpha=0.8)
plt.title('Y-axis sun sensor signal')
plt.xlabel('Time (s)')
plt.legend()

# Plot z-axis signals
plt.subplot(3, 1, 3)
plt.plot(time, noisy_signal[:, 2], label='Noisy signal', alpha=0.6)
plt.plot(time, filtered_signal[:, 2], label='Filtered signal', alpha=0.8)
plt.title('Z-axis sun sensor signal')
plt.xlabel('Time (s)')
plt.legend()

# Adjust layout and show the plot
plt.tight_layout()
plt.show()

In [ ]:
# Gyro filtering example

kernel = AleasimKernel(dt=sample_period)
adyn = AttitudeDynamicsModel(kernel)
gyro = GyroSensor('gyro_sensor', kernel, int(1 / sample_period))
gyro_filter = LowPassFilter('bessel', int(1 / sample_period), order=order, cutoff_freq=cutoff_freq)

kernel.add_model(adyn)
kernel.add_model(gyro, create_shared_mem=True)

gyro.power_on()

noisy_signal = []
filtered_signal = []

for i in range(signal_length):
    # Advance the kernel
    kernel.step()
    gyro.measure()

    # Get the latest x, y, z readings from the state array
    noisy_signal_temp = gyro.state_array[-1, 3:6]
    filered_signal_temp = gyro_filter.filter_output(gyro.state_array[:, 3:6])

    # Store the signals
    noisy_signal.append(noisy_signal_temp)
    filtered_signal.append(filered_signal_temp)

# Convert lists to numpy arrays for easier plotting
noisy_signal = np.array(noisy_signal)
filtered_signal = np.array(filtered_signal)

time = np.arange(signal_length) * sample_period

# noisy_signal = noisy_signal[:-20]
# filtered_signal = filtered_signal[:-20]
# time = time[:-20]

# Plot noisy and filtered signals for each axis (x, y, z)
plt.figure(figsize=(12, 6))

# Plot x-axis signals
plt.subplot(3, 1, 1)
plt.plot(time, noisy_signal[:, 0], label='Noisy signal', alpha=0.6)
plt.plot(time, filtered_signal[:, 0], label='Filtered signal', alpha=0.8)
plt.title('X-axis gyro signal')
plt.xlabel('Time (s)')
plt.legend()

# Plot y-axis signals
plt.subplot(3, 1, 2)
plt.plot(time, noisy_signal[:, 1], label='Noisy signal', alpha=0.6)
plt.plot(time, filtered_signal[:, 1], label='Filtered signal', alpha=0.8)
plt.title('Y-axis gyro signal')
plt.xlabel('Time (s)')
plt.legend()

# Plot z-axis signals
plt.subplot(3, 1, 3)
plt.plot(time, noisy_signal[:, 2], label='Noisy signal', alpha=0.6)
plt.plot(time, filtered_signal[:, 2], label='Filtered signal', alpha=0.8)
plt.title('Z-axis gyro signal')
plt.xlabel('Time (s)')
plt.legend()

# Adjust layout and show the plot
plt.tight_layout()
plt.show()

## Bode plot comparison

In [ ]:
from scipy import signal

fs = 1 / sample_period     # Sampling frequency in Hz

# Moving Average Filter Coefficients
num_avg = (1 / window_size) * np.ones(window_size)
den_avg = [1]

# Butterworth Filter Coefficients
num_butter, den_butter = signal.butter(
    N=order, Wn=cutoff_freq, btype='low', analog=False, fs=fs
)

# Bessel Filter Coefficients
num_bessel, den_bessel = signal.bessel(
    N=order, Wn=cutoff_freq, btype='low', analog=False, fs=fs, norm='phase'
)

# Compute frequency responses
w_avg, h_avg = signal.freqz(num_avg, den_avg, fs=fs)
w_butter, h_butter = signal.freqz(num_butter, den_butter, fs=fs)
w_bessel, h_bessel = signal.freqz(num_bessel, den_bessel, fs=fs)

# Plotting
plt.figure(figsize=(14, 8))

# Magnitude plot
plt.subplot(2, 1, 1)
plt.semilogx(w_avg, 20 * np.log10(np.abs(h_avg)), color='blue', label='Moving Average')
plt.semilogx(w_butter, 20 * np.log10(np.abs(h_butter)), color='green', label='Butterworth')
plt.semilogx(w_bessel, 20 * np.log10(np.abs(h_bessel)), color='orange', label='Bessel')
plt.title('Bode Plot Comparison')
plt.ylabel('Magnitude (dB)')
plt.grid(True)
plt.legend()
plt.xlim(1, fs / 2)
plt.axvline(cutoff_freq, color='red')  # cutoff frequency

# Phase plot
plt.subplot(2, 1, 2)
plt.semilogx(w_avg, np.unwrap(np.angle(h_avg)) * 180 / np.pi, color='blue', label='Moving Average')
plt.semilogx(w_butter, np.unwrap(np.angle(h_butter)) * 180 / np.pi, color='green', label='Butterworth')
plt.semilogx(w_bessel, np.unwrap(np.angle(h_bessel)) * 180 / np.pi, color='orange', label='Bessel')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Phase (degrees)')
plt.grid(True)
plt.legend()
plt.xlim(1, fs / 2)
plt.axvline(cutoff_freq, color='red')  # cutoff frequency

plt.tight_layout()
plt.show()